---

copyright:
  years: 2021, 2022
lastupdated: "2022-03-21"

keywords: quantum, Qiskit, runtime, near time compute

subcollection: quantum-computing

---

{{site.data.keyword.attribute-definition-list}}


# Sampler primitive
{: #program-sampler}

Learn how to use the Sampler primitive.
{: shortdesc}

## Overview
{: #sampler-overview}

The Sampler primitive returns sample distributions (quasi-probabilities) generated by given circuits executed on the target backend.  

The maximum execution time is 10000 seconds (2.78 hours).

## Parameters
{: #sampler-parameters}

Sample distributions generated by given circuits executed on the target backend.
- **Input parameters for Sampler**:
    - **run_options**:
        - **Type**: object
        - **Description**: A collection of key-value pairs identifying the execution options, such as shots.
        - **Required**: False
    - **circuit_indices**:
        - **Type**: array
        - **Description**: Indexes of the circuits to evaluate.
        - **Required**: True
    - **parameter_values**
        - **Type**: array
        - **Description**: Concrete parameters to be bound.
        - **Required**: False
- **Input parameters for Sampler**:
    - **skip_transpilation**:
        - **Description**: Skip circuit transpilation. The default value is False.
        - **Required**: False.   
    - **circuits**:
        - **Description**: A QuantumCircuit or list of QuantumCircuits. The circuits can be parameterized.
        - **Required**: True
    - **parameters**
        - **Type**: array
        - **Description**: Parameters of the quantum circuits.
        - **Required**: False       
    - **options**        
        - **Description**: Additional parameters, such as the backend to use.
        - **Required**: False
- **Returns**:
   - **metadata**:
        - **Type**: array
        - **Description**: Additional metadata.  
        - **Required**: False
   - **quasi_dists**:
       - **Description**: Quasidistribution for each circuit.
       - **Type**: array
       - **Required**: False

## Example
{: #program-sampler-sample-code}

```Python
from qiskit_ibm_runtime import QiskitRuntimeService, Sampler
from qiskit import QuantumCircuit

service = QiskitRuntimeService(channel="ibm_cloud", token="<api-token>", instance="<IBM Cloud CRN>")

bell = QuantumCircuit(2)
bell.h(0)
bell.cx(0, 1)
bell.measure_all()

# executes a Bell circuit
with Sampler(circuits=bell, service=service, options={ "backend": "ibmq_qasm_simulator" }) as sampler:
    result = sampler(circuit_indices=[0], shots=1024)
    print(result)
```
{: codeblock}

```text
SamplerResult(quasi_dists=[{'00': 0.4873046875, '11': 0.5126953125}], metadata=[{'header_metadata': None, 'shots': 1024}])
```

## Next steps
{: #next-steps}

- Use the [Getting started guide](/docs/quantum-computing?topic=quantum-computing-quickstart) to create an instance and run your first job.
- Review [Get started with the Sampler primitive](/docs/quantum-computing?topic=quantum-computing-example-sampler) for a step-by-step example of using this primitive.
- Use Qiskit [tutorials](https://qiskit.org/documentation/tutorials.html){: external} to learn how to create circuits with Qiskit.
